In [ ]:
import tensorflow.keras as tfk
import numpy as np

In [ ]:
path = tfk.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt',
)
print(path)

In [ ]:
text = open(path, 'r').read().lower()
text_len = len(text)
print(f'text length: {text_len}')

In [ ]:
max_len = 60
step = 3
sequences = [] # 保存所有的序列
labels = [] # 保存所有的标签
for i in range(0, text_len - max_len, step):
    sequences.append(text[i: i + max_len]) # 数据
    labels.append(text[i + max_len]) # 标签
print('sequences count:', len(sequences))
print('labels count:', len(labels))

In [ ]:
chars = sorted(list(set(text)))
print('unique chars:', len(chars))

In [ ]:
chars_dict = {char: i for i, char in enumerate(chars)}
print(chars_dict)

In [ ]:
# 向量化
X = np.zeros((len(sequences), max_len, len(chars)), dtype=np.bool)
y = np.zeros((len(labels), len(chars)), dtype=np.bool)

# one-hot 编码
for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        X[i, t, chars_dict[char]] = 1

for i, label in enumerate(labels):
    y[i, chars_dict[label]] = 1

X.shape, y.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.optimizers import RMSprop

In [ ]:
model = Sequential()
model.add(Input(shape=(max_len, len(chars))))
model.add(LSTM(128))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
model.summary()

In [ ]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64') # 类型转换
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds) # 归一化
    probas = np.random.multinomial(1, preds, 1) # 多项式采样，随机采样
    return np.argmax(probas) # 返回概率值最大的字符

In [ ]:
import random

model.fit(X, y, batch_size=32, epochs=1)
for epoch in range(1, 10):
    print('epoch:', epoch)
    start_idx = random.randint(0, len(text) - max_len - 1)
    generate_text = text[start_idx: start_idx + max_len]
    print('random generated text:', generate_text)

    for t in [0.2, 0.5, 1.0, 1.2]:
        print('temperature:', t)
        print('generating text:', generate_text)
        for i in range(400):
            sampled = np.zeros((1, max_len, len(chars)))
            for j, char in enumerate(generate_text):
                sampled[0, j, chars_dict[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, t)
            next_char = chars[next_index]

            generate_text += next_char
            generate_text = generate_text[1:]

            print(generate_text)
